In [ ]:
# USER: Change the path below to point to your local copy of ImageD11:

import os

home_dir = !echo $HOME
home_dir = str(home_dir[0])

# USER: You can change this location if you want

id11_code_path = os.path.join(home_dir, "Code/ImageD11")

import sys

sys.path.insert(0, id11_code_path)

os.environ['OMP_NUM_THREADS']='1'

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib ipympl

import ImageD11.sinograms.point_by_point
import ImageD11.sinograms.dataset
import ImageD11.sinograms.properties
import ImageD11.columnfile

import ImageD11.nbGui.nb_utils as utils

In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/data/visitor/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/James/nb_testing"

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_250um"

In [ ]:
# desination of H5 files

dset_file = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_file)

print(ds)
print(ds.shape)

In [ ]:
# This assumes you've only just segmented, so we need to add these things to the dataset:

# USER: specify the path to the parameter file

par_file = os.path.join(processed_data_root_dir, '../../../SCRIPTS/James/S3DXRD/Fe_refined.par')
e2dx_file = os.path.join(processed_data_root_dir, '../../CeO2/e2dx_E-08-0173_20231127.edf')
e2dy_file = os.path.join(processed_data_root_dir, '../../CeO2/e2dy_E-08-0173_20231127.edf')

# add them to the dataset

ds.parfile = par_file
ds.e2dxfile = e2dx_file
ds.e2dyfile = e2dy_file

In [ ]:
# Import the 2D columnfile, or make it if it doesn't exist

if os.path.exists(ds.col2dfile):
    cf_2d = ImageD11.columnfile.colfile_from_hdf(ds.col2dfile)
else:
    # Import 2D peaks, make a spatially corrected columnfile, save it
    peaks_table = ImageD11.sinograms.properties.pks_table.load(ds.pksfile)

    # Grab the 2d peak centroids
    peaks_2d = peaks_table.pk2d(ds.omega, ds.dty)
    cf_2d = utils.tocolf(peaks_2d)

    # save the 2D peaks to file so we don't have to spatially correct them again
    ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)

In [ ]:
# filter the columnfile to discard weak peaks

cf_2d.filter(cf_2d.Number_of_pixels > 5)

In [ ]:
pbp_object = ImageD11.sinograms.point_by_point.PBP(ds.parfile,
                                                    ds,
                                                    hkl_tol=0.01,
                                                    fpks=0.9,
                                                    ds_tol=0.005,
                                                    etacut=0.1,
                                                    ifrac=1./300,
                                                    cosine_tol=np.cos(np.radians(90 - 0.25)),
                                                    y0=-7.985,
                                                    symmetry="cubic",
                                                    foridx=[0, 1, 2, 3, 4],
                                                    forgen=[0, 1],
                                                    uniqcut=0.9)

In [ ]:
pbp_object.setpeaks(cf_2d)

In [ ]:
fig, ax = pbp_object.iplot()

In [ ]:
ds.pbpfile = os.path.join(ds.analysispath, ds.dsname + '_pbp_index.pbp')

In [ ]:
pbp_object.point_by_point(ds.pbpfile, loglevel=3)

In [ ]:
ds.save()